정수 리터럴과 덧셈만 있는 아주 간단한 산술식을 하스켈으로 다뤄보자.

In [1]:
:opt no-lint                          -- linter 끄기
{-# LANGUAGE ScopedTypeVariables #-}  -- 추가 언어 기능 설정

\noindent
문법구조
$\hspace{-2ex}\begin{array}{ll}
& n \in \texttt{Int} \\
& e \in \texttt{Expr} ~::=~ n ~\mid~ e + e
\end{array}$는 데이터 타입으로 선언하면 된다.

In [2]:
data Expr = I Int          -- n
          | Add Expr Expr  -- e + e
       deriving (Eq,Ord,Show) -- Eq 같은지, Ord 순서 비교, Show 출력 가능

작은걸음 의미구조\vspace*{-3ex}
$$
\frac{~ ~}{~ n_1 + n_2 \longmapsto n ~}
{\scriptstyle(\,n\;\equiv\;n_1\,+\,n_2\,)}
\quad
\frac{~ e_1 \longmapsto e_1' ~}{~ e_1 + e_2 \longmapsto e_1' + e_e ~}
\quad
\frac{~ e_1 \longmapsto e_1' ~}{~ e_1 + e_2 \longmapsto e_1' + e_e ~}
$$
는 어떻게? TODO TODO TODO

# 작은걸음 의미구조 검산기

In [3]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2   =  e1 ...> e1'  -- 좌변에 변수 e2 중복
Add e1 e2 ...> Add e1  e2'  =  e2 ...> e2'  -- 좌변에 변수 e1 중복
_         ...> _            =  False  -- 다른 경우에는 진행 불가

: 

In [4]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2'  =  e1 ...> e1' && e2 == e2'  ||
                               e2 ...> e2' && e1 == e1'
_         ...> _            =  False  -- 다른 경우에는 진행 불가

In [5]:
:type (...>)

(...>) :: Expr -> Expr -> Bool

In [6]:
Add (I 1) (I 2) ...> I 3  -- 검산 성공
Add (I 1) (I 2) ...> I 4  -- 검산 실패

True

False

In [7]:
I 1 `Add` I 2 ...> I 3
I 1 `Add` I 2 ...> I 4

True

False

In [8]:
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 3 `Add` I 4)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 1 `Add` I 2)(I 7)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 7) -- 한꺼번에 2걸음

True

True

False

# 작은걸음 의미구조 계산기
비결정적인 작은걸음 관계($\longmapsto\;\subset\;\texttt{Expr}\times\texttt{Expr}$)를
함수($f_{\longmapsto}$)로 만드는 방법은 모든 가능한 다음 단계를 한꺼번에 모아놓은 집합을
결과값으로 하면 된다. 즉, 식($e\in\texttt{Expr}$) 하나에
식의 집합($\{e_1',e_2',\ldots,e_k'\}\in 2^\texttt{Expr}$) 하나를 대응시키는
함수 $f_{\longmapsto} : \texttt{Expr} \to 2^\texttt{Expr}$를
$f_{\longmapsto}(e) = \{\,e' \mid e\longmapsto e'\,\}$로 정의하면 된다.
이 정의는 단순명료하지만 작은걸음 의미구조의 계산기를 작성하는 데 적합하지 않다.
식 $e$의 문법구조를 따라 정의된 작은걸음 의미구조처럼 $f_{\longmapsto}$를
귀납적(inductive)으로 다음과 같이 정의하면 프로그램으로 옮기기에 좋다.
단, 마지막 등식에서 $e_1+e_2$는 위의 경우들과 겹치지 않는 형태($e_1+e_2\neq n_1+n_2$)라고 가정한다.
\vspace*{-2ex}
\begin{align*}
& f_{\longmapsto} ~ : ~ \texttt{Expr} \to 2^\texttt{Expr} \\
& f_{\longmapsto}(n)             &=~& \{\} \\
& f_{\longmapsto}(n_1 + n_2)     &=~& \{ n \} \qquad(n \equiv n_1 + n_2) \\
& f_{\longmapsto}(\,e_1 + e_2\,) &=~& \{ e_1' + e_2 \mid e_1\longmapsto e_1' \} \cup \{ e_1 + e_2' \mid e_2\longmapsto e_2'\}
\end{align*}

In [9]:
step :: Expr -> [Expr] -- 리스트로 집합을 표현
step n                 = []
step (I n1 `Add` I n2) = [ I n ]  where n = n1 + n2
step (Add e1 e2)       = [ Add e1' e2  | e1' <- step e1 ] ++
                         [ Add e1  e2' | e2' <- step e2 ]

## 하스켈의 리스트

In [10]:
:type 'a'           -- 문자
:type ['a','b','c'] -- 문자 리스트 = 문자열(String)
:type (&&)          -- 함수
:type [(&&), (||)]  -- 함수 리스트

'a' :: Char

['a','b','c'] :: [Char]

(&&) :: Bool -> Bool -> Bool

[(&&), (||)] :: [Bool -> Bool -> Bool]

In [11]:
[1,2,3] ++ [4,5,6]    -- 정수 리스트 이어붙이기

[1,2,3,4,5,6]

In [12]:
[I 1, Add (I 1) (I 2)] ++ [Add (I 3) (I 4), I 3]

[I 1,Add (I 1) (I 2),Add (I 3) (I 4),I 3]

In [1]:
:type (++)

(++) :: forall a. [a] -> [a] -> [a]

리스트 조건제시법

In [13]:
-- 리스트 조건제시법